In [3]:
import os
import requests
from dotenv import load_dotenv
import base64
import json
import numpy as np
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
from utils.sf_conn import Config
from utils.assistant_api import Spotify_Assistant
from utils.prompt import PROMPT
import time
load_dotenv()

True

In [ ]:
sf_config = Config()
create_conn = sf_config.create_sf_conn()

In [ ]:
cursor = create_conn.cursor()
query = "SELECT * FROM MAIN_SPOTIFY"
spotify_df = cursor.execute(query).fetch_pandas_all()
spotify_df.head(3)

In [ ]:
spotify_csv = spotify_df.to_csv('assistant_data/spotify_main_file')

In [ ]:
def wait_until_run_status_completed(assistant, run_id):
    start_time = time.time() # starting a timer in case, the while loop gets stuck
    while True:
        (status := assistant.run_status(run_id))
        if status == "completed":
            break
        elif status in {"in_progress", "queued"}:
            cur_time = time.time()
            time_elasped = cur_time - start_time
            if time_elasped >= 600: # break the loop after it's been running for 10 mins
                break
            continue
        else:
            break
    assert status == "completed"

In [ ]:
# def upload_csv_file(file_path):
#     api_key =  os.getenv('OPENAI_API_KEY')
#     openai.api_key = api_key
#     with open(file_path, "rb") as file:
#         uploaded_file = openai.files.create(
#             file=file,
#             purpose='assistants'
#         )
#     return uploaded_file.id

In [4]:
assistant = Spotify_Assistant(assistant_id='asst_afQAdlE40WllyVRI7mDZBjha')
upload_csv = assistant.upload_csv_file('assistant_data/spotify_main_file')
print("------------upload_csv--------------------------------")
print(upload_csv)
files = assistant.show_filed_uplaoded()
print("---------Show Files in Assistant-------------------")
print(files)
print("---------Delete Files in assistant---------------------")
# delete_files = assistant.delete_file()
# print(delete_files)


------------upload_csv--------------------------------
file-OeQpRRWniFizmrAvRw7qvls0
---------Show Files in Assistant-------------------
['file-OeQpRRWniFizmrAvRw7qvls0']
---------Delete Files in assistant---------------------


In [5]:
delete_files = assistant.delete_file()
delete_files

FileDeleteResponse(id='file-OeQpRRWniFizmrAvRw7qvls0', deleted=True, object='assistant.file.deleted')


TypeError: 'FileDeleteResponse' object is not subscriptable

In [ ]:
for i in files:
    print(i.id)

In [ ]:
! pip list

In [ ]:
import os

import snowflake.connector as sfc
from snowflake.connector.pandas_tools import write_pandas
import json



def create_sf_conn(
            wh='COMPUTE_WH',
            r='DATA_ENGINEER',
            keep_alive=False):
    conn = sfc.connect(
            user = os.getenv("SF_USERNAME"),
            password = os.getenv("SF_PASSWORD"),
            account = os.getenv('SF_ACCOUNT'),
            warehouse = wh,
            role = r,
            database = 'analytics_justin', 
            schema = 'spotify_schema', 
            client_session_keep_alive=keep_alive)
    return conn

In [ ]:
con = create_sf_conn()
cursor = con.cursor()
execute = cursor.execute('SELECT * from main_spotify')
execute.fetch_pandas_all()
